In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import joblib


# Scaling
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import RobustScaler 
from sklearn.preprocessing import MinMaxScaler 

# Train Test Split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_validate, cross_val_score

# Feature Selector

# Models
# import torch
# import torch.nn as nn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Keras
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.models import Sequential, Model
from keras.layers import Dense, ReLU, Dropout, InputLayer,LeakyReLU

# Optimizer
from keras.optimizers import Adam

# Kfold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

print('Packages imported...')

from sklearn.metrics import classification_report, confusion_matrix


Packages imported...


In [2]:
df = pd.read_csv('DataFinal.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,10844,10845,10846,10847,10848,10849,10850,10851,10852,10853
0,7.0,5.0,4.0,4.0,3.0,3.0,3.0,2.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
X = df.drop('10853', axis=1)
y_cat = tf.keras.utils.to_categorical(df['10853'])
y = df['10853']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.3, random_state=1)
y_train_nn = tf.keras.utils.to_categorical(y_train)
y_test_nn = tf.keras.utils.to_categorical(y_test)

In [5]:
from sklearn.feature_selection import SelectKBest, chi2

def chi2_feature_selection(X_train, X_test, n):
    selector = SelectKBest(chi2, k=n)
    X_train_new = selector.fit_transform(X_train, y_train)
    X_test_new = selector.transform(X_test)
    return {"X_train": X_train_new, "X_test": X_test_new}

In [6]:
CHI2_data = {}
CHI2_data[2500] = chi2_feature_selection(X_train, X_test, 2500)

In [10]:
def NN(X_train, X_test, feature_selection_option, n = 0):
    print("Loại trích chọn đặc trưng: ", feature_selection_option)
    # Optimize
    adam = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Model
    model = keras.Sequential([
    keras.layers.Dense(64, input_shape = (X_train.shape[1],), activation='sigmoid'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='sigmoid'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='sigmoid'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(17, activation='softmax')])

    model.compile(optimizer=adam,
                    loss='categorical_crossentropy',
                    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    model.fit(X_train, y_train_nn, epochs=8, batch_size=8)

    loss, accuracy, precision, recall = model.evaluate(X_test, y_test_nn)
    print("Loss of NN: " + str(loss))
    print("Accuracy of NN: " + str(accuracy))
    print("Precision of NN: " + str(precision))
    print("Recall of NN: " + str(recall))

    # model.save("nn_{}_model.h5".format(feature_selection_option + "_" + str(n)))
    y_pred = model.predict(X_test)
    cr = classification_report(y_test_nn.argmax(axis=1), y_pred.argmax(axis=1))
    print(cr)



In [17]:
# instantiating the object
def SVM(X_train, X_test, feature_selection_option, n = 0):
    print("Loại trích chọn đặc trưng: ", feature_selection_option)
    svm = SVC(kernel='linear')

    # fitting the model
    svm.fit(X_train,y_train)

    # predicting the values
    y_pred = svm.predict(X_test)

    # Confusion matrix
    conf = confusion_matrix(y_test, y_pred)
    # print ("Confusion Matrix : \n", conf)

    # printing the test accuracy
    svm_acc = accuracy_score(y_test, y_pred)
    print("Accuracy:", svm_acc)
    svm_precision = precision_score(y_test, y_pred, average="weighted")
    print("Precision: ", svm_precision)
    svm_recall = recall_score(y_test, y_pred, average="weighted")
    print("Recall: ", svm_recall)

    # joblib.dump(svm, "svm_{}_model.joblib".format(feature_selection_option + "_" + str(n)))
    cr = classification_report(y_test, y_pred)
    print(cr)

In [15]:
def KNN(X_train, X_test, feature_selection_option, n = 0):
    print("Loại trích chọn đặc trưng: ", feature_selection_option)
    # instantiating the object and fitting
    knn = KNeighborsClassifier(n_neighbors = 7)  

    # fitting the model
    knn.fit(X_train,y_train)

    # predicting the values
    y_pred = knn.predict(X_test)

    # Confusion matrix
    conf = confusion_matrix(y_test, y_pred)
    # print ("Confusion Matrix : \n", conf)

    # printing the test accuracy
    knn_acc = accuracy_score(y_test, y_pred)
    print("Accuracy score của KNN là: ", knn_acc)
    knn_precision = precision_score(y_test, y_pred, average="weighted")
    print("Precision score của KNN là: ", knn_precision)
    knn_recall = recall_score(y_test, y_pred, average="weighted")
    print("Recall score của KNN là: ", knn_recall)

    # joblib.dump(knn, "knn_{}_model.joblib".format(feature_selection_option + "_" + str(n)))
    cr = classification_report(y_test, y_pred)
    print(cr)


Run

In [11]:
NN(X_train, X_test, "None")

Loại trích chọn đặc trưng:  None
Epoch 1/8
1031/1031 [==============================] - 9s 7ms/step - loss: 1.8059 - accuracy: 0.5207 - precision_1: 0.9676 - recall_1: 0.0507
Epoch 2/8
1031/1031 [==============================] - 7s 7ms/step - loss: 0.7416 - accuracy: 0.8680 - precision_1: 0.9580 - recall_1: 0.6222
Epoch 3/8
1031/1031 [==============================] - 7s 6ms/step - loss: 0.3732 - accuracy: 0.9356 - precision_1: 0.9660 - recall_1: 0.8795
Epoch 4/8
1031/1031 [==============================] - 7s 6ms/step - loss: 0.2410 - accuracy: 0.9540 - precision_1: 0.9712 - recall_1: 0.9297
Epoch 5/8
1031/1031 [==============================] - 7s 7ms/step - loss: 0.1657 - accuracy: 0.9716 - precision_1: 0.9812 - recall_1: 0.9571
Epoch 6/8
1031/1031 [==============================] - 7s 6ms/step - loss: 0.1313 - accuracy: 0.9749 - precision_1: 0.9809 - recall_1: 0.9665
Epoch 7/8
1031/1031 [==============================] - 7s 7ms/step - loss: 0.1046 - accuracy: 0.9781 - precision_1:

In [18]:
SVM(X_train, X_test, "None")

Loại trích chọn đặc trưng:  None


In [16]:
KNN(X_train, X_test, "None")

Loại trích chọn đặc trưng:  None
Accuracy score của KNN là:  0.8124469589816125
Precision score của KNN là:  0.833067123078088
Recall score của KNN là:  0.8124469589816125
              precision    recall  f1-score   support

         1.0       0.89      0.83      0.86       323
         2.0       0.92      0.81      0.86       296
         5.0       0.87      0.65      0.74       315
         6.0       0.92      0.81      0.86       298
         7.0       0.68      0.79      0.73       264
         8.0       0.75      0.76      0.75       287
        11.0       0.78      0.82      0.80       286
        12.0       0.91      0.76      0.83       298
        13.0       0.83      0.70      0.76       291
        14.0       0.88      0.93      0.91       289
        15.0       0.96      0.94      0.95       295
        16.0       0.58      0.97      0.72       293

    accuracy                           0.81      3535
   macro avg       0.83      0.81      0.81      3535
weighted avg    